In [21]:
import pandas as pd
import duckdb
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Set up display options
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_rows', 50)

# Connect to database
db_path = "../data/premierleague_raw.duckdb"
conn = duckdb.connect(db_path)
print(f"✅ Connected to: {db_path}")
print("🏈 Premier League Analytics - Using Exact Schema Column Names")

✅ Connected to: ../data/premierleague_raw.duckdb
🏈 Premier League Analytics - Using Exact Schema Column Names


In [22]:
# Connect to database
db_path = "../data/premierleague_raw.duckdb"
conn = duckdb.connect(db_path)
print(f"✅ Connected to: {db_path}")

# Get current season status
fixture_info = conn.execute("""
SELECT 
    MAX(current_through_gameweek) as current_gameweek,
    COUNT(*) as total_fixtures,
    SUM(CASE WHEN is_completed THEN 1 ELSE 0 END) as completed_fixtures
FROM raw_fixtures
""").fetchdf().iloc[0]

print(f"🏈 Season Status:")
print(f"  Current gameweek: {fixture_info['current_gameweek']}")
print(f"  Completed fixtures: {fixture_info['completed_fixtures']}/{fixture_info['total_fixtures']}")

# Quick table counts
table_counts = conn.execute("""
SELECT 
    'player_standard' as table_name, COUNT(*) as rows FROM player_standard
UNION ALL
SELECT 
    'squad_standard' as table_name, COUNT(*) as rows FROM squad_standard  
UNION ALL
SELECT 
    'opponent_standard' as table_name, COUNT(*) as rows FROM opponent_standard
""").fetchdf()
print(f"\n📊 Key Table Counts:")
print(table_counts)

✅ Connected to: ../data/premierleague_raw.duckdb
🏈 Season Status:
  Current gameweek: 4.0
  Completed fixtures: 30.0/380.0

📊 Key Table Counts:
          table_name  rows
0    player_standard   370
1     squad_standard    20
2  opponent_standard    20


In [23]:
print("⚽ TOP GOALSCORERS (through Gameweek 4):")
goalscorers = conn.execute("""
SELECT 
    Player,
    Squad,
    Pos,
    "Performance Gls" as Goals,
    "Performance Ast" as Assists,
    "Performance G+A" as Goals_Plus_Assists,
    "Playing Time 90s" as Games_90s
FROM player_standard 
WHERE "Performance Gls" > 0
ORDER BY 
    "Performance Gls" DESC, 
    "Performance G+A" DESC
LIMIT 10
""").fetchdf()
print(goalscorers)

print("\n🎯 TOP ASSIST PROVIDERS:")
assists = conn.execute("""
SELECT 
    Player,
    Squad,
    Pos,
    "Performance Ast" as Assists,
    "Performance Gls" as Goals,
    "Performance G+A" as Goals_Plus_Assists,
    "Playing Time 90s" as Games_90s
FROM player_standard 
WHERE "Performance Ast" > 0
ORDER BY 
    "Performance Ast" DESC,
    "Performance G+A" DESC
LIMIT 10
""").fetchdf()
print(assists)

⚽ TOP GOALSCORERS (through Gameweek 4):
            Player            Squad    Pos  Goals  Assists  \
0   Erling Haaland  Manchester City     FW      3        0   
1       João Pedro          Chelsea  MF,FW      2        2   
2   Jaidon Anthony          Burnley  FW,MF      2        1   
3     Hugo Ekitike        Liverpool     FW      2        1   
4   Enzo Fernández          Chelsea     MF      2        1   
5      Richarlison        Tottenham     FW      2        1   
6  Antoine Semenyo      Bournemouth  MF,FW      2        1   
7   Jurriën Timber          Arsenal     DF      2        1   
8  Viktor Gyökeres          Arsenal     FW      2        0   
9    Lucas Paquetá         West Ham     MF      2        0   

   Goals_Plus_Assists  Games_90s  
0                   3        2.8  
1                   4        2.8  
2                   3        2.9  
3                   3        2.5  
4                   3        2.9  
5                   3        2.6  
6                   3        3.0

In [24]:
print("🎯 MOST CREATIVE PLAYERS (Key Passes + Expected Assists):")
creative_players = conn.execute("""
SELECT 
    ps.Player,
    ps.Squad,
    ps.Pos,
    pp.KP as Key_Passes,
    pp.xAG as Expected_Assists,
    ps."Performance Ast" as Actual_Assists,
    pp."90s" as Games_90s,
    ROUND(CAST(pp.KP as FLOAT) / pp."90s", 2) as Key_Passes_per_90
FROM player_passing pp
JOIN player_standard ps ON pp.Player = ps.Player
WHERE 
    pp.KP > 0 
    AND pp."90s" >= 2
ORDER BY 
    CAST(pp.KP as FLOAT) / pp."90s" DESC
LIMIT 12
""").fetchdf()
print(creative_players)

🎯 MOST CREATIVE PLAYERS (Key Passes + Expected Assists):
                   Player           Squad    Pos  Key_Passes  \
0         Bruno Fernandes  Manchester Utd     MF          12   
1             Anton Stach    Leeds United     MF          10   
2       James Ward-Prowse        West Ham     MF           8   
3            Bryan Mbeumo  Manchester Utd     MF           9   
4   Kiernan Dewsbury-Hall         Everton     MF           8   
5            David Brooks     Bournemouth  MF,FW           6   
6             Mason Mount  Manchester Utd     MF           5   
7        Mikkel Damsgaard       Brentford     MF           5   
8          Mohammed Kudus       Tottenham     FW           7   
9           Harvey Barnes   Newcastle Utd     FW           5   
10             Saša Lukić          Fulham     MF           7   
11          Jack Grealish         Everton     FW           5   

    Expected_Assists  Actual_Assists  Games_90s  Key_Passes_per_90  
0                0.7               0     

In [25]:
print("🎯 CLINICAL FINISHERS (Goals per Shot on Target):")
clinical_finishers = conn.execute("""
SELECT 
    Player,
    Squad,
    Pos,
    "Standard SoT%" as Shot_Accuracy_Pct,
    "Standard G/SoT" as Goals_per_SoT,
    "Standard Gls" as Goals,
    "Standard Sh" as Total_Shots,
    "Standard SoT" as Shots_on_Target,
    "90s" as Games_90s
FROM player_shooting 
WHERE 
    "Standard Sh" >= 8
    AND "Standard SoT" > 0
ORDER BY 
    "Standard G/SoT" DESC,
    "Standard SoT%" DESC
LIMIT 10
""").fetchdf()
print(clinical_finishers)

# xG Over/Underperformers
print("\n⚽ xG PERFORMANCE vs ACTUAL GOALS:")
xg_performance = conn.execute("""
SELECT 
    ps.Player,
    ps.Squad,
    ps.Pos,
    ps."Performance Gls" as Goals,
    psh."Expected xG" as Expected_Goals,
    psh."Expected G-xG" as Goals_minus_xG,
    psh."90s" as Games_90s,
    ROUND(CAST(psh."Expected G-xG" as FLOAT) / psh."90s", 2) as xG_diff_per_90
FROM player_standard ps
JOIN player_shooting psh ON ps.Player = psh.Player
WHERE 
    psh."Expected xG" > 1.0
    AND psh."90s" >= 2
ORDER BY 
    psh."Expected G-xG" DESC
""").fetchdf()

print("🔥 OVERPERFORMERS (scoring above expected):")
print(xg_performance.head(8))
print("\n❄️ UNDERPERFORMERS (scoring below expected):")
print(xg_performance.tail(5))

🎯 CLINICAL FINISHERS (Goals per Shot on Target):
            Player            Squad    Pos  Shot_Accuracy_Pct  Goals_per_SoT  \
0       João Pedro          Chelsea  MF,FW               37.5           0.67   
1      Richarlison        Tottenham     FW               37.5           0.67   
2   Erling Haaland  Manchester City     FW               35.7           0.60   
3  Antoine Semenyo      Bournemouth  MF,FW               40.0           0.50   
4        Evanilson      Bournemouth     FW               25.0           0.50   
5     Bryan Mbeumo   Manchester Utd     MF               50.0           0.25   
6    Matheus Cunha   Manchester Utd  FW,MF               62.5           0.00   
7   Anthony Gordon    Newcastle Utd     FW               30.0           0.00   
8    Ollie Watkins      Aston Villa     FW               25.0           0.00   
9     David Brooks      Bournemouth  MF,FW               22.2           0.00   

   Goals  Total_Shots  Shots_on_Target  Games_90s  
0      2          

In [26]:
print("🚀 MOST PROGRESSIVE PLAYERS (Carries + Passes):")
progressive_players = conn.execute("""
SELECT 
    ps.Player,
    ps.Squad,
    ps.Pos,
    ps."Progression PrgC" as Progressive_Carries,
    pp.PrgP as Progressive_Passes,
    CAST(ps."Progression PrgC" as FLOAT) + CAST(pp.PrgP as FLOAT) as Total_Progressive,
    ps."Playing Time 90s" as Games_90s,
    ROUND((CAST(ps."Progression PrgC" as FLOAT) + CAST(pp.PrgP as FLOAT)) / ps."Playing Time 90s", 2) as Progressive_per_90
FROM player_standard ps
JOIN player_passing pp ON ps.Player = pp.Player
WHERE 
    ps."Playing Time 90s" >= 2
    AND (ps."Progression PrgC" > 0 OR pp.PrgP > 0)
ORDER BY 
    Total_Progressive DESC
LIMIT 15
""").fetchdf()
print(progressive_players)

🚀 MOST PROGRESSIVE PLAYERS (Carries + Passes):
                Player            Squad Pos  Progressive_Carries  \
0           Alex Iwobi           Fulham  FW                   19   
1      Elliot Anderson  Nott'ham Forest  MF                    6   
2   Morgan Gibbs-White  Nott'ham Forest  MF                    7   
3              Murillo  Nott'ham Forest  DF                    2   
4        Sandro Tonali    Newcastle Utd  MF                    8   
5      Bruno Fernandes   Manchester Utd  MF                    6   
6          Anton Stach     Leeds United  MF                    7   
7          Djed Spence        Tottenham  DF                   10   
8        Lucas Paquetá         West Ham  MF                    2   
9           Pedro Neto          Chelsea  FW                   14   
10       Neco Williams  Nott'ham Forest  DF                    9   
11     Bruno Guimarães    Newcastle Utd  MF                    3   
12     Youri Tielemans      Aston Villa  MF                    7   
1

In [27]:
print("🛡️ DEFENSIVE WORKHORSES (Tackles + Interceptions per 90):")
defenders = conn.execute("""
SELECT 
    ps.Player,
    ps.Squad,
    ps.Pos,
    pd."Tackles Tkl" as Tackles,
    pd.Int as Interceptions,
    pd."Tackles TklW" as Tackles_Won,
    CAST(pd."Tackles Tkl" as FLOAT) + CAST(pd.Int as FLOAT) as Defensive_Actions,
    pd."90s" as Games_90s,
    ROUND((CAST(pd."Tackles Tkl" as FLOAT) + CAST(pd.Int as FLOAT)) / pd."90s", 2) as Def_Actions_per_90
FROM player_standard ps
JOIN player_defense pd ON ps.Player = pd.Player
WHERE 
    pd."90s" >= 2 
    AND (ps.Pos LIKE '%DF%' OR ps.Pos LIKE '%MF%')
    AND (pd."Tackles Tkl" > 0 OR pd.Int > 0)
ORDER BY 
    Def_Actions_per_90 DESC
LIMIT 12
""").fetchdf()
print(defenders)

🛡️ DEFENSIVE WORKHORSES (Tackles + Interceptions per 90):
              Player            Squad    Pos  Tackles  Interceptions  \
0     Moisés Caicedo          Chelsea     MF       12              8   
1    Rayan Aït-Nouri  Manchester City     DF        9              5   
2        Tyler Adams      Bournemouth     MF        9              7   
3    Tyrick Mitchell   Crystal Palace  DF,MF       14              1   
4   Reinildo Mandava       Sunderland     DF       10              5   
5              André           Wolves     MF        7              2   
6    Elliot Anderson  Nott'ham Forest     MF       12              1   
7      Marcos Senesi      Bournemouth     DF        6              7   
8        Yasin Ayari         Brighton     MF        5              4   
9    Trevoh Chalobah          Chelsea     DF        7              5   
10     Omar Alderete       Sunderland     DF        5              5   
11   Maxence Lacroix   Crystal Palace     DF        6              6   

    T

In [28]:
print("🎨 BEST DRIBBLERS (Take-On Success):")
dribblers = conn.execute("""
SELECT 
    ps.Player,
    ps.Squad,
    ps.Pos,
    pp."Touches Touches" as Touches,
    pp."Take-Ons Att" as TakeOns_Attempted,
    pp."Take-Ons Succ" as TakeOns_Successful,
    pp."Take-Ons Succ%" as TakeOn_Success_Pct,
    pp."90s" as Games_90s,
    ROUND(CAST(pp."Take-Ons Succ" as FLOAT) / pp."90s", 2) as Successful_TakeOns_per_90
FROM player_standard ps
JOIN player_possession pp ON ps.Player = pp.Player
WHERE 
    pp."90s" >= 2
    AND pp."Take-Ons Att" >= 10
ORDER BY 
    pp."Take-Ons Succ%" DESC,
    pp."Take-Ons Succ" DESC
LIMIT 12
""").fetchdf()
print(dribblers)

🎨 BEST DRIBBLERS (Take-On Success):
              Player            Squad    Pos  Touches  TakeOns_Attempted  \
0     Yankuba Minteh         Brighton     FW       96                 15   
1    Estêvão Willian          Chelsea     FW       90                 16   
2         Pedro Neto          Chelsea     FW      144                 10   
3         Oscar Bobb  Manchester City  FW,MF      134                 15   
4   Marcus Tavernier      Bournemouth     MF      135                 11   
5    Antoine Semenyo      Bournemouth  MF,FW      128                 14   
6      Morgan Rogers      Aston Villa  FW,MF      106                 15   
7       Jarrod Bowen         West Ham     FW      124                 15   
8     Mohammed Kudus        Tottenham     FW      120                 23   
9      Iliman Ndiaye          Everton     FW       99                 14   
10     Jack Grealish          Everton     FW      115                 19   
11          Ola Aina  Nott'ham Forest     DF      20

In [29]:
print("🎯 AERIAL DOMINANCE BY POSITION:")
aerial_dominance = conn.execute("""
SELECT 
    ps.Pos,
    ROUND(AVG(CAST(pm."Aerial Duels Won%" as FLOAT)), 1) as Avg_Aerial_Win_Pct,
    ROUND(AVG(CAST(pm."Aerial Duels Won" as FLOAT)), 1) as Avg_Aerials_Won,
    COUNT(*) as Players,
    ROUND(SUM(CAST(pm."Aerial Duels Won" as FLOAT)), 0) as Total_Aerials_Won
FROM player_standard ps
JOIN player_misc pm ON ps.Player = pm.Player
WHERE 
    ps."Playing Time 90s" >= 2 
    AND pm."Aerial Duels Won" > 0
GROUP BY 
    ps.Pos
ORDER BY 
    Avg_Aerial_Win_Pct DESC
""").fetchdf()
print(aerial_dominance)

print("\n🏆 TOP INDIVIDUAL AERIAL PLAYERS:")
top_aerial = conn.execute("""
SELECT 
    ps.Player,
    ps.Squad,
    ps.Pos,
    pm."Aerial Duels Won" as Aerials_Won,
    pm."Aerial Duels Won%" as Aerial_Win_Pct,
    pm."90s" as Games_90s,
    ROUND(CAST(pm."Aerial Duels Won" as FLOAT) / pm."90s", 2) as Aerials_Won_per_90
FROM player_standard ps
JOIN player_misc pm ON ps.Player = pm.Player
WHERE 
    pm."90s" >= 2
    AND pm."Aerial Duels Won" >= 15
ORDER BY 
    pm."Aerial Duels Won%" DESC,
    pm."Aerial Duels Won" DESC
LIMIT 10
""").fetchdf()
print(top_aerial)

🎯 AERIAL DOMINANCE BY POSITION:
     Pos  Avg_Aerial_Win_Pct  Avg_Aerials_Won  Players  Total_Aerials_Won
0     GK                91.7              1.4       14               20.0
1     DF                62.9              6.0       67              405.0
2  DF,MF                60.8              2.3        4                9.0
3     FW                47.8              4.9       27              132.0
4     MF                45.6              3.1       32               98.0
5  MF,FW                42.5              6.7        3               20.0
6  FW,MF                32.2              2.0        4                8.0
7  DF,FW                25.0              1.0        1                1.0

🏆 TOP INDIVIDUAL AERIAL PLAYERS:
             Player           Squad Pos  Aerials_Won  Aerial_Win_Pct  \
0   Virgil van Dijk       Liverpool  DF           17            77.3   
1  Matthijs de Ligt  Manchester Utd  DF           17            58.6   

   Games_90s  Aerials_Won_per_90  
0        3.0    

In [30]:
print("⚽ TEAM ATTACKING vs DEFENSIVE PERFORMANCE (FIXED):")
team_performance = conn.execute("""
SELECT 
    ss.Squad,
    ss."Performance Gls" as Goals_Scored,
    os."Performance Gls" as Goals_Conceded,
    ss."Performance Gls" - os."Performance Gls" as Goal_Difference,
    ss."Expected xG" as xG_For,
    os."Expected xG" as xG_Against,
    ROUND(ss."Expected xG" - os."Expected xG", 2) as xG_Difference,
    ROUND(ss."Performance Gls" / 4.0, 2) as Goals_per_Game,
    ROUND(os."Performance Gls" / 4.0, 2) as Goals_Conceded_per_Game
FROM squad_standard ss
JOIN opponent_standard os ON ss.Squad = REPLACE(os.Squad, 'vs ', '')
ORDER BY 
    Goal_Difference DESC
""").fetchdf()
print(team_performance)

⚽ TEAM ATTACKING vs DEFENSIVE PERFORMANCE (FIXED):
              Squad  Goals_Scored  Goals_Conceded  Goal_Difference  xG_For  \
0           Chelsea             7               1                6     6.4   
1           Arsenal             6               1                5     4.4   
2         Liverpool             8               4                4     3.4   
3         Tottenham             5               1                4     3.6   
4    Crystal Palace             4               1                3     4.4   
5           Everton             5               3                2     4.3   
6        Sunderland             5               3                2     3.0   
7   Manchester City             5               4                1     5.8   
8       Bournemouth             4               4                0     4.5   
9          Brighton             3               4               -1     6.1   
10          Burnley             4               5               -1     3.1   
11           

In [31]:
print("🔍 COMPLETE PLAYER PROFILES (5-table join):")
complete_profiles = conn.execute("""
SELECT 
    ps.Player,
    ps.Squad,
    ps.Pos,
    ps."Performance Gls" as Goals,
    ps."Performance Ast" as Assists,
    pp."Total Cmp%" as Pass_Accuracy,
    pp.KP as Key_Passes,
    psh."Standard Sh" as Shots,
    psh."Expected xG" as xG,
    po."Touches Touches" as Touches,
    po."Take-Ons Succ%" as TakeOn_Success_Pct,
    ps."Playing Time 90s" as Games_90s
FROM player_standard ps
JOIN player_passing pp ON ps.Player = pp.Player
JOIN player_shooting psh ON ps.Player = psh.Player  
JOIN player_possession po ON ps.Player = po.Player
WHERE 
    ps."Playing Time 90s" >= 3
ORDER BY 
    ps."Performance G+A" DESC
LIMIT 15
""").fetchdf()
print("Multi-table join combining standard + passing + shooting + possession:")
print(complete_profiles)

print("\n💡 This shows the complexity - joining 4 tables for basic player analysis!")


🔍 COMPLETE PLAYER PROFILES (5-table join):
Multi-table join combining standard + passing + shooting + possession:
                   Player            Squad    Pos  Goals  Assists  \
0         Antoine Semenyo      Bournemouth  MF,FW      2        1   
1              Cody Gakpo        Liverpool     FW      1        2   
2   Kiernan Dewsbury-Hall          Everton     MF      1        1   
3           Mohamed Salah        Liverpool     FW      1        1   
4           Lucas Paquetá         West Ham     MF      2        0   
5             Josh Cullen          Burnley     MF      1        1   
6                  Thiago        Brentford     FW      2        0   
7       Tijjani Reijnders  Manchester City     MF      1        1   
8            Ismaila Sarr   Crystal Palace  MF,FW      2        0   
9   El Hadji Malick Diouf         West Ham     DF      0        2   
10        Elliot Anderson  Nott'ham Forest     MF      0        1   
11             Marc Guéhi   Crystal Palace     DF      1  

In [32]:
print("\n" + "="*60)
print("🔍 KEY INSIGHTS FROM EXPLORATION")
print("="*60)

# Get some interesting stats
top_scorer = conn.execute("""
SELECT Player, Squad, "Performance Gls" 
FROM player_standard 
ORDER BY "Performance Gls" DESC 
LIMIT 1
""").fetchone()

most_creative = conn.execute("""
SELECT pp.Player, ps.Squad, pp.KP 
FROM player_passing pp 
JOIN player_standard ps ON pp.Player = ps.Player 
WHERE pp."90s" >= 2
ORDER BY CAST(pp.KP as FLOAT) / pp."90s" DESC 
LIMIT 1
""").fetchone()

print("\n🎯 KEY FINDINGS:")
print(f"- Top scorer: {top_scorer[0]} ({top_scorer[1]}) with {top_scorer[2]} goals")
print(f"- Most creative: {most_creative[0]} ({most_creative[1]}) with {most_creative[2]} key passes")

print("\n✅ DATA VALIDATION:")
print("- All queries now use exact schema column names")
print("- Multi-table joins working correctly")
print("- Rich football metrics available for analysis")

print("\n📊 ANALYTICS LAYER PRIORITIES:")
print("1. Consolidate player data (eliminate 5-table joins)")
print("2. Standardize per-90 metrics consistently")
print("3. Create player role classifications")
print("4. Build team tactical profiles")

print("\n🎯 READY FOR ANALYTICS LAYER DEVELOPMENT!")

# Close connection
conn.close()
print("🔚 Analysis complete - database connection closed")


🔍 KEY INSIGHTS FROM EXPLORATION

🎯 KEY FINDINGS:
- Top scorer: Erling Haaland (Manchester City) with 3 goals
- Most creative: Bruno Fernandes (Manchester Utd) with 12 key passes

✅ DATA VALIDATION:
- All queries now use exact schema column names
- Multi-table joins working correctly
- Rich football metrics available for analysis

📊 ANALYTICS LAYER PRIORITIES:
1. Consolidate player data (eliminate 5-table joins)
2. Standardize per-90 metrics consistently
3. Create player role classifications
4. Build team tactical profiles

🎯 READY FOR ANALYTICS LAYER DEVELOPMENT!
🔚 Analysis complete - database connection closed
